# Counting in Arbitrary Bases
** November 2017 **

** Andrew Riberio @ [AndrewRib.com](http://www.andrewrib.com) **

A demo of my hefty counter which can count in an arbitrary base. 

Port from my C++ Version: https://github.com/Andrewnetwork/breaking-the-code/blob/master/AndrewWork/Server/HeftyCounter.cpp

## Ratonale 
I wrote a botnet that orchestrated the distributed solution of problems. At its core was a counting mechanism. The first conception for the system was to brute force the cracking of passwords-- the simplest use case we thought of for distributed computing. In order to do this, we would count in the base of the length of the password character lexicon, and every number in that base could be easily translated to a possible password in that character set. More importantly, ranges of numbers would represent password combinations. These password ranges would be sent to computers in the botnet whereupon the range of passwords would be tried against some locked file -- I designed it so that it would send the password range to an executable, making it more robust than I'm making it seem. 

This implementation can count in any arbitrary base and is extremely efficient. The size of the counter is only limited by the memory capacity of the host computer, as we numpy ndarray’s as the core data structure. 

A more full featured version of this code will be released as a python package in the near future. 

In [128]:
import numpy as np

class HeftyCounter:
    def __init__(self,length,base):
        self.base        = base
        self.counterSize = length
        self.counter     = np.zeros(length)
        
    def __repr__(self):
        return str(self.counter)
    
    def b10(self):
        return self.bn(10)
    
    def bn(self,n):
        a = HeftyCounter(1,n)
        idx = self.counterSize - 1
        countr = 0
        while idx >= 0 and self.counter[idx] != 0:
            r = self.counter[idx]*(self.base**countr)
            a.increment(r)
            countr += 1
            idx -= 1
        return a
        
    def increment(self, quantity ):

        if quantity != 0:
            remainder         = quantity
            additionRemainder = 0
            counterIndex      = 0
            additionTemp      = 0
            digitCount        = 1
            cond              = True

            #Convert quantity to the counters base and add it to the counter at the same time.
            while cond:
                counterIndex               = self.counterSize - digitCount
                additionTemp               = (self.counter[counterIndex] + (remainder % self.base) + additionRemainder)
                self.counter[counterIndex] = additionTemp % self.base
                additionRemainder          = int(additionTemp) // int(self.base)
                remainder                  = remainder // self.base

                if (additionRemainder > 0 or remainder > 0) and digitCount >= self.counterSize :
                    #Overflow.
                    self.counter      = np.insert(self.counter,0,0)
                    self.counterSize += 1

                digitCount += 1

                cond = remainder > 0 or additionRemainder > 0

## Basics 
Let's count in base 30. We start off with a blank slate. 

In [129]:
cntr = HeftyCounter(1,30)
cntr.counter

array([ 0.])

Our counter will dynamically expand as needed. Let's add a million to our counter. 

In [130]:
cntr.increment(1000000000)
cntr

[  1.  11.   4.  17.   1.   3.  10.]

This is 1 million in base 30. We can confirm this by converting the counter to base 10.

In [132]:
cntr.b10()

[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

We can easily convert this counter to any base. Let's see what it is in base 242 ( bet you haven't worked in that base!):

In [136]:
cntr.bn(242)

[  70.  135.   81.   98.]